In [2]:
from constants import JQL, FIELDS, BASE_URL, JIRA_DOMAIN,EMAIL, MAX_RESULTS, MODULE_DEVS, VALID_STATUSES_BT, MAIL_BT_MAP, DAILY_BT_HOURS,MIN_BT_PROJECT_RATIO,PROJECT_MAP, DEFAULT_END_DATE, DEFAULT_END_DATE_with_timezone, DEFAULT_START_DATE,DEFAULT_START_DATE_with_timezone
from token_hidden import API_TOKEN

In [3]:
import os
import time
import csv
import math
import requests
from requests.auth import HTTPBasicAuth
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from typing import List, Optional, Dict

# ===================== CONFIG =====================
SITE = "https://team-1583163151751.atlassian.net"
EMAIL = os.getenv("JIRA_EMAIL") or EMAIL     # deja tu variable existente
API_TOKEN = os.getenv("JIRA_API_TOKEN") or API_TOKEN

AUTH = HTTPBasicAuth(EMAIL, API_TOKEN)
HEADERS = {"Accept": "application/json"}
TIMEOUT = 30
MAX_WORKERS = int(os.getenv("MAX_WORKERS", "8"))  # concurrencia para fetch de issues

# Endpoints
ISSUE_GET = SITE + "/rest/api/3/issue/{key}?expand=changelog"
AGILE_BOARDS = SITE + "/rest/agile/1.0/board"
AGILE_BOARD_ISSUES = SITE + "/rest/agile/1.0/board/{boardId}/issue"

# ===================== HTTP helpers =====================
SESSION = requests.Session()

def http_get(url: str, params: Dict = None, ok_404: bool = False):
    """GET con reintentos y backoff exponencial; maneja 429/5xx."""
    backoff = 0.5
    for attempt in range(6):
        r = SESSION.get(url, headers=HEADERS, auth=AUTH, params=params, timeout=TIMEOUT)
        # 2xx
        if 200 <= r.status_code < 300:
            return r
        # 404 permitido
        if ok_404 and r.status_code == 404:
            return r
        # 429 o 5xx -> retry con backoff + Retry-After si viene
        if r.status_code in (429, 500, 502, 503, 504):
            wait = r.headers.get("Retry-After")
            wait_s = float(wait) if wait and wait.isdigit() else backoff
            time.sleep(wait_s)
            backoff = min(backoff * 2, 8.0)
            continue
        # otros códigos -> error inmediato
        r.raise_for_status()
    r.raise_for_status()
    return r  # never reached

# ===================== UTIL =====================
def parse_jira_dt(s: str) -> datetime:
    # Ej: 2025-09-30T09:50:38.240-0300
    return datetime.strptime(s, "%Y-%m-%dT%H:%M:%S.%f%z")

def extract_time_between_statuses(issue_json: dict, from_status: str, to_status: str):
    ch = issue_json.get("changelog", {}) or {}
    histories = ch.get("histories", []) or []
    from_date, to_date = None, None
    for h in histories:
        created = h.get("created")
        for item in h.get("items", []) or []:
            if item.get("field") == "status":
                if item.get("toString") == from_status and not from_date:
                    from_date = created
                if item.get("toString") == to_status and not to_date:
                    to_date = created
    if from_date and to_date:
        dt_from = parse_jira_dt(from_date)
        dt_to = parse_jira_dt(to_date)
        hours = (dt_to - dt_from).total_seconds() / 3600
        return {"from": from_date, "to": to_date, "hours": round(hours, 2)}
    return None

def fetch_issue_with_changelog(key: str) -> Optional[dict]:
    url = ISSUE_GET.format(key=key)
    r = http_get(url, ok_404=True)
    if r.status_code == 404:
        return None
    return r.json()

# ===================== RUTA A: AGILE API =====================
def get_first_board_for_project(project_key: str) -> Optional[int]:
    # probamos kanban
    r = http_get(AGILE_BOARDS, params={"projectKeyOrId": project_key, "type": "kanban"})
    data = r.json()
    if data.get("values"):
        return data["values"][0]["id"]
    # probamos scrum
    r = http_get(AGILE_BOARDS, params={"projectKeyOrId": project_key, "type": "scrum"})
    data = r.json()
    if data.get("values"):
        return data["values"][0]["id"]
    return None

def list_issue_keys_via_board(board_id: int, jql: str | None = None, page_size: int = 50) -> List[str]:
    start_at = 0
    keys = []
    while True:
        params = {"startAt": start_at, "maxResults": page_size, "fields": "key"}
        if jql:
            params["jql"] = jql  # p.ej. created >= -90d
        r = http_get(AGILE_BOARD_ISSUES.format(boardId=board_id), params=params)
        data = r.json()
        batch = [i["key"] for i in (data.get("issues") or [])]
        if not batch:
            break
        keys.extend(batch)
        start_at += len(batch)
        total = data.get("total", start_at)
        if start_at >= total:
            break
        time.sleep(0.1)
    return keys

# ===================== RUTA B: RANGO DESDE SEMILLA =====================
def probe_key_exists(project_key: str, number: int) -> bool:
    key = f"{project_key}-{number}"
    url = ISSUE_GET.format(key=key)
    r = http_get(url, ok_404=True)
    if r.status_code == 200:
        return True
    if r.status_code == 404:
        return False
    r.raise_for_status()
    return False

def discover_keys_around_seed(project_key: str, seed_number: int, span_each_side: int = 500, concurrency: int = MAX_WORKERS) -> List[str]:
    """
    Escanea IT-(seed-span .. seed+span) en paralelo, guardando solo las existentes (200).
    """
    numbers = list(range(max(1, seed_number - span_each_side), seed_number + span_each_side + 1))
    keys_found = []
    with ThreadPoolExecutor(max_workers=concurrency) as ex:
        fut_map = {ex.submit(probe_key_exists, project_key, n): n for n in numbers}
        for fut in as_completed(fut_map):
            n = fut_map[fut]
            try:
                if fut.result():
                    keys_found.append(f"{project_key}-{n}")
            except requests.HTTPError:
                time.sleep(0.3)  # bajamos ritmo si hubo error
    keys_found.sort(key=lambda k: int(k.split("-")[1]))
    return keys_found

# ===================== EXPORT =====================
def row_from_issue(issue_json: dict, cycle: dict) -> dict:
    fields = issue_json.get("fields", {}) or {}
    assignee = fields.get("assignee") or {}
    issue_type = fields.get("issuetype") or {}
    return {
        "issueKey": issue_json.get("key"),
        "issueType": issue_type.get("name"),
        "assignee": assignee.get("displayName"),
        "summary": fields.get("summary"),
        "from_date": cycle["from"],
        "to_date": cycle["to"],
        "hours": cycle["hours"],
    }

def export_cycle_times(keys: List[str], from_status: str, to_status: str, csv_out: str, concurrency: int = MAX_WORKERS):
    rows = []

    def process_key(k: str) -> Optional[dict]:
        issue = fetch_issue_with_changelog(k)
        if not issue:
            return None
        cyc = extract_time_between_statuses(issue, from_status, to_status)
        if not cyc:
            return None
        return row_from_issue(issue, cyc)

    with ThreadPoolExecutor(max_workers=concurrency) as ex:
        futs = [ex.submit(process_key, key) for key in keys]
        done = 0
        for fut in as_completed(futs):
            row = fut.result()
            if row:
                rows.append(row)
            done += 1
            if done % 25 == 0:
                print(f"…procesados {done}/{len(keys)}")

    rows.sort(key=lambda r: (r["to_date"] or "", r["issueKey"] or ""))
    with open(csv_out, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["issueKey", "issueType", "assignee", "summary", "from_date", "to_date", "hours"])
        w.writeheader()
        w.writerows(rows)
    print(f"✅ CSV generado: {csv_out} ({len(rows)} filas)")

# ===================== MAIN =====================
if __name__ == "__main__":
    PROJECT = "IT"
    FROM_STATUS = "Pendiente de estimación"
    TO_STATUS = "Estimada"
    CSV_OUT = "cycle_times.csv"

    # ---- Opción A: Agile API (recomendada si hay board) ----
    keys: List[str] = []
    board_id = get_first_board_for_project(PROJECT)
    if board_id:
        # TIP: podés acotar con jql="created >= -90d" para menos volumen
        keys = list_issue_keys_via_board(board_id, jql=None, page_size=50)
        print(f"🔑 (Agile) Keys obtenidas: {len(keys)}")

    # ---- Opción B: Semilla (si no hay board/permiso) ----
    if not keys:
        SEED = 21773     # confirmaste que IT-21773 existe
        SPAN = 2000      # +/- 1000 alrededor de la semilla
        keys = discover_keys_around_seed(PROJECT, SEED, span_each_side=SPAN, concurrency=MAX_WORKERS)
        print(f"🔑 (Seed) Keys obtenidas: {len(keys)} (rango IT-{SEED-SPAN}..IT-{SEED+SPAN})")

    if not keys:
        print("⚠️ No se encontraron issues. Revisá permisos, project y el board/rango.")
    else:
        export_cycle_times(keys, FROM_STATUS, TO_STATUS, csv_out=CSV_OUT, concurrency=MAX_WORKERS)


🔑 (Seed) Keys obtenidas: 2016 (rango IT-19773..IT-23773)
…procesados 25/2016
…procesados 50/2016
…procesados 75/2016
…procesados 100/2016
…procesados 125/2016
…procesados 150/2016
…procesados 175/2016
…procesados 200/2016
…procesados 225/2016
…procesados 250/2016
…procesados 275/2016
…procesados 300/2016
…procesados 325/2016
…procesados 350/2016
…procesados 375/2016
…procesados 400/2016
…procesados 425/2016
…procesados 450/2016
…procesados 475/2016
…procesados 500/2016
…procesados 525/2016
…procesados 550/2016
…procesados 575/2016
…procesados 600/2016
…procesados 625/2016
…procesados 650/2016
…procesados 675/2016
…procesados 700/2016
…procesados 725/2016
…procesados 750/2016
…procesados 775/2016
…procesados 800/2016
…procesados 825/2016
…procesados 850/2016
…procesados 875/2016
…procesados 900/2016
…procesados 925/2016
…procesados 950/2016
…procesados 975/2016
…procesados 1000/2016
…procesados 1025/2016
…procesados 1050/2016
…procesados 1075/2016
…procesados 1100/2016
…procesados 1125/